In [9]:
import numpy as np
from numpy import load
import cv2
import matplotlib
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier

from google.colab import drive
drive.mount('/content/drive')
# make path
path = '/content/drive/My Drive/ECSE416_ComputerVision/A3/'
print(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ECSE416_ComputerVision/A3/


## 1. Image Classification using RF and SVM

SVM

In [10]:
# Step 1 - load the images and labels 
train_images = np.load(path+"flower_subset.npz")["train_images"] 
train_labels = np.load(path+"flower_subset.npz")["train_labels"] 
test_images = np.load(path+"flower_subset.npz")["test_images"] 
test_labels = np.load(path+"flower_subset.npz")["test_labels"]

# #Step 2 - Assess the size of out images and labels
print("train images dimentions: ",train_images.shape)
print("train labels dimentions: ",train_labels.shape)
print("test images dimentions: ",test_images.shape)
print("test labels dimentions: ",test_labels.shape)

# Step 3 - Resize the images from 128x128 to 64x64
test_images_resize = []
train_images_resize = []

for i in range(1556):
  train_images_temp = cv2.resize(train_images[i], (0,0), fx=0.5, fy=0.5)
  train_images_resize.append(train_images_temp)

train_images_resize = np.stack(train_images_resize)

for i in range(90):
  test_images_temp = cv2.resize(test_images[i], (0,0), fx=0.5, fy=0.5)
  test_images_resize.append(test_images_temp)

test_images_resize = np.stack(test_images_resize)

print('\n'+"Resized train images dimentions: ",train_images_resize.shape)
print("Resized test images dimentions: ",test_images_resize.shape)

#Step 4 - extract HoG features
img_size = (64, 64) 
cell_size = (8, 8) 
block_size = (4, 4) 
nbins = 4 

hog = cv2.HOGDescriptor(_winSize=(img_size[1] // cell_size[1] * cell_size[1],
                                  img_size[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

features_train = []
features_test = []

for i in range(train_images_resize.shape[0]):
    features_train.append(hog.compute((train_images_resize[i]*255).astype(np.uint8)).reshape(1, -1))
features_train = np.vstack(features_train)

for i in range(test_images_resize.shape[0]):
    features_test.append(hog.compute((test_images_resize[i]*255).astype(np.uint8)).reshape(1, -1))
features_test = np.vstack(features_test)

#Step 5 - create the classifier
clf = svm.SVC(gamma ='auto', C=1.)

#Step 6 - train classifier on training set
clf.fit(features_train,train_labels)

#Step 7 - test the model, Predict labels for the test set
predict = clf.predict(features_test)

print('\n'+"Predicted Label:", predict)
print("Actual Label:",test_labels)

# Step 8 - calculate classifiction accurary
correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print('\n'+"Classification Accuracy: " + str((correct/len(predict))*100) + "%")


train images dimentions:  (1556, 128, 128)
train labels dimentions:  (1556,)
test images dimentions:  (90, 128, 128)
test labels dimentions:  (90,)

Resized train images dimentions:  (1556, 64, 64)
Resized test images dimentions:  (90, 64, 64)

Predicted Label: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]

Classification Accuracy: 11.11111111111111%


### Acuracy of greater than 50%

In [11]:
#Step 5 - create the classifier
clf = svm.SVC(gamma ='auto', C=100)

#Step 6 - train classifier on training set
clf.fit(features_train,train_labels)

#Step 7 - test the model, Predict labels for the test set
predict = clf.predict(features_test)

print('\n'+"Predicted Label:", predict)
print("Actual Label:",test_labels)

# Step 8 - calculate classifiction accurary
correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print('\n'+"Classification Accuracy: " + str((correct/len(predict))*100) + "%")


Predicted Label: [0 0 0 0 0 3 8 0 0 0 0 7 5 1 1 1 1 6 1 1 1 2 3 2 2 2 2 2 2 2 7 3 3 2 3 3 3
 1 3 6 4 4 4 4 4 4 2 4 0 4 0 2 5 6 5 5 1 1 4 5 1 1 1 7 6 6 6 1 5 6 1 1 0 0
 7 3 1 7 4 1 0 8 8 1 8 4 0 8 0 4]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]

Classification Accuracy: 55.55555555555556%


# Random Forest

In [12]:
# Step 2 - create a Random forest classifier 
clf = RandomForestClassifier(n_estimators=10, max_depth=5, criterion='entropy')

# Step 3 - train classifier on training set
clf.fit(features_train,train_labels)

# Step 4 - test the model, Predict labels for the test set

predict = clf.predict(features_test)

print("Predicted Label:", predict)
print("Actual Label:",test_labels)


# Step 5 - calculate classifiction accurary
correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print('\n'+"Classification Accuracy: " + str((correct/len(predict))*100) + "%")

Predicted Label: [4 4 4 0 4 6 4 1 4 4 4 4 7 1 1 0 1 3 4 2 4 2 3 4 2 6 2 2 2 4 7 6 3 2 1 2 2
 3 3 6 4 4 1 4 4 4 1 4 4 4 0 4 3 2 3 4 2 7 1 1 6 1 3 4 2 6 6 1 3 2 1 1 4 0
 4 7 7 1 0 1 4 3 8 1 4 4 4 8 4 8]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]

Classification Accuracy: 31.11111111111111%


### Acuracy of greater than 50%

In [7]:
# Step 2 - create a Random forest classifier 
clf = RandomForestClassifier(n_estimators=100, max_depth=10, criterion='entropy')

# Step 3 - train classifier on training set
clf.fit(features_train,train_labels)

# Step 4 - test the model, Predict labels for the test set

predict = clf.predict(features_test)

print("Predicted Label:", predict)
print("Actual Label:",test_labels)


# Step 5 - calculate classifiction accurary
correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print('\n'+"Classification Accuracy: " + str((correct/len(predict))*100) + "%")

Predicted Label: [0 0 0 0 0 6 8 0 0 0 0 7 1 1 1 1 1 6 1 7 7 2 3 2 2 2 2 2 2 2 1 3 3 2 1 2 3
 1 3 6 4 4 4 4 4 4 4 4 4 4 4 4 5 1 1 4 5 1 1 7 6 7 1 1 2 6 6 1 3 6 7 7 0 0
 7 7 1 7 4 0 4 8 8 0 8 8 1 8 0 4]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]

Classification Accuracy: 57.77777777777777%


Compare results of SVM and RF classifiers. Which one provides better results? Experiment training both classifiers with a range of random states (try different random values for the argument ‘random state’) and measure classification accuracy of the test set. Which classifier is more stable or robust to the change in random state?

*Comparing SVM and RF with the given classifiers RF achieved a better classification acuaracy at 37.78% compared to SVM at 11.11%. RF is more roubust to change in the random state as the random sate was changed from 20 to 300 and there was no change in SVM classification accuracy. Where as a change from 20 to 300 in RF leads to a increase from 32.2% to 40% in classification accuracy.*

In [35]:
# SVM random_state=20
clf = svm.SVC(gamma ='auto', C=10, random_state=20)

clf.fit(features_train,train_labels)

predict = clf.predict(features_test)

correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print("Classification Accuracy: " + str((correct/len(predict))*100) + "%")

Classification Accuracy: 30.0%


In [36]:
# SVM random_state=300
clf = svm.SVC(gamma ='auto', C=10, random_state=300)

clf.fit(features_train,train_labels)

predict = clf.predict(features_test)

correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print("Classification Accuracy: " + str((correct/len(predict))*100) + "%")

Classification Accuracy: 30.0%


In [37]:
# RF random_state=20
clf = RandomForestClassifier(n_estimators=10, max_depth=10, criterion='entropy', random_state=20)

clf.fit(features_train,train_labels)

predict = clf.predict(features_test)

correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print("Classification Accuracy: " + str((correct/len(predict))*100) + "%")

Classification Accuracy: 32.22222222222222%


In [38]:
# RF random_state=300
clf = RandomForestClassifier(n_estimators=10, max_depth=10, criterion='entropy', random_state=300)

clf.fit(features_train,train_labels)

predict = clf.predict(features_test)

correct = 0
for i in range(len(predict)):
    if(predict[i] == test_labels[i]):
        correct+=1

print("Classification Accuracy: " + str((correct/len(predict))*100) + "%")

Classification Accuracy: 40.0%
